## Setting

In [33]:
from torchvision import datasets as ds
from torch.utils.data import DataLoader
from torchvision import transforms as ts
import torchvision as tv
import torch
import torch.nn as nn
import torch.nn.functional as func
import math
import numpy as np
from torch.autograd import Variable
from torch import optim
from matplotlib import pyplot as plt
import torch.backends.cudnn as cudnn
import timm
from PIL import Image
from tqdm import tqdm
import random
import os

%config Completer.use_jedi = False

In [2]:
def setseed(seednum = 20):
    torch.manual_seed(seednum)
    torch.cuda.manual_seed(seednum)
    torch.cuda.manual_seed_all(seednum)
    np.random.seed(seednum)
    cudnn.benchmark = False
    cudnn.deterministic = True
    random.seed(seednum)

In [3]:
setseed(35)

In [4]:
cuda_available = torch.cuda.is_available()
device = torch.device("cuda" if cuda_available else "cpu")
device

device(type='cuda')

## Dataset

In [5]:
transform = ts.Compose(
    [
        ts.ToTensor(),
        ts.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

In [6]:
train_set = ds.CIFAR10(root='../data/', train=True, transform=transform, download=True)
val_set = tv.datasets.CIFAR10(root='../data/', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=125, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=125, shuffle=False, num_workers=0)

In [8]:
def make_test_list(data_dir):
    
    test_img_list = list()
    
    files = os.listdir(data_dir)
    
    for i in range(10):
        img_file = data_dir + "/" + files[i]
        img = os.listdir(img_file)
        for j in range(200):
            img_path = data_dir + "/" +  files[i] + "/" + img[j]
            
            test_img_list.append(img_path)
    return test_img_list

In [9]:
img_list = make_test_list("../Statistical_Deep_Image_2021/Statistical_Deep_Image")

In [10]:
def make_test_label(label_list):
    
    test_label_list = list()
    
    for i in range(10):
        for j in range(200):
            test_label_list.append(i)
        
    return test_label_list

In [11]:
label = os.listdir("../Statistical_Deep_Image_2021/Statistical_Deep_Image")
img_label_list = make_test_label(label)

In [12]:
class testset(torch.utils.data.Dataset):
    def __init__(self, img_list, img_label_list, transform):
        self.file_list = img_list
        self.labels = img_label_list
        self.transform = transform
        
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = Image.open(img_path)
        img_transformed = self.transform(img)
        label = self.labels[index]
        return img_transformed, label

In [13]:
test_set = testset(img_list = img_list,
                  img_label_list = img_label_list,
                  transform = transform)

In [14]:
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 125, shuffle=False, num_workers=0)

In [15]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## Model

In [34]:
class LeNet(nn.Module):  
        def __init__(self):  
            super().__init__()  
            self.conv=nn.Conv2d(3,20,5,1)  
            self.conv1=nn.Conv2d(20,50,5,1)  
            self.fully=nn.Linear(5*5*50,500)  
            self.dropout=nn.Dropout(0.5)   
            self.fully1=nn.Linear(500,10)  
        def forward(self,y):  
            y=func.relu(self.conv(y))  
            y=func.max_pool2d(y,2,2)  
            y=func.relu(self.conv1(y))  
            y=func.max_pool2d(y,2,2) 
            # Reshaping the output 
            y=y.view(-1,5*5*50) 
            # Apply the relu activation function
            y=func.relu(self.fully(y))   
            y=self.dropout(y)  
            y=self.fully1(y)    
            return y      

In [28]:
model = LeNet()

In [29]:
model = model.to(device)

In [30]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

## Train

In [39]:
def train(num_epoch):
    best_accuracy = 0.0 
    
    for epoch in tqdm(range(num_epoch)):

        running_train_loss = 0.0
        running_val_loss = 0.0
        accuracy = 0
        total = 0

        for i, data in enumerate(train_loader, 0) :
            inputs, labels = data[0].to(device), data[1].to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_train_loss += loss.item()
        
        with torch.no_grad():
            model.eval()
            for i, data in enumerate(val_loader, 0):
                inputs, labels = data[0].to(device), data[1].to(device)
                predicted_outputs = model(inputs)
                val_loss = criterion(predicted_outputs, labels)
                
                _, predicted = torch.max(predicted_outputs, 1) 
                
                running_val_loss += val_loss.item()
                total += outputs.size(0)
                accuracy += (predicted == labels).sum().item()
                
        train_loss_per_epoch = running_train_loss / len(train_loader)
        val_loss = running_val_loss/len(val_loader)
        accuracy = (100 * accuracy / total)     
        
        if accuracy > best_accuracy:
            torch.save(model.state_dict(), 'LeNet_weights.pth')
            best_accuracy = accuracy
            
        print('epoch: %d' %(epoch+1), ' train_loss: %.3f' %train_loss_per_epoch, ' val_loss: %.4f' %val_loss, ' Accuracy: %d %%' % (accuracy))
            

In [42]:
train(60)

  2%|▏         | 1/60 [00:10<10:41, 10.88s/it]

epoch: 1  train_loss: 0.188  val_loss: 1.2354  Accuracy: 68 %


  3%|▎         | 2/60 [00:20<09:43, 10.06s/it]

epoch: 2  train_loss: 0.172  val_loss: 1.2711  Accuracy: 68 %


  3%|▎         | 2/60 [00:24<11:52, 12.29s/it]


KeyboardInterrupt: 

## Test

In [43]:
model.load_state_dict(torch.load('LeNet_weights.pth'))
model.eval()

LeNet(
  (conv): Conv2d(3, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv1): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fully): Linear(in_features=1250, out_features=500, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fully1): Linear(in_features=500, out_features=10, bias=True)
)

In [44]:
correct = 0
total = 0

with torch.no_grad():
    for data in tqdm(test_loader):
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print('Accuracy: %d %%' % (100 * correct / total))

100%|██████████| 16/16 [00:00<00:00, 17.85it/s]

Accuracy: 27 %
